# Ejercicios embeddings de oraciones

In [2]:
!pip install spacy
!pip install scikit-learn
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 16.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Ejercicio 1

Desarrolla una función que tome dos argumentos: una oración objetivo y una lista de oraciones. Esta función debe calcular y devolver la oración de la lista que es más similar a la oración objetivo, basándose en la medida de la similitud coseno. Además, la función también debe retornar el puntaje de similitud obtenido por esta oración.

In [3]:
import spacy

# Cargar el modelo de lenguaje en español
nlp = spacy.load('es_core_news_md')

def encontrar_frase_similar(frase_objetivo, lista_de_frases):
    # Crear un embedding para la frase objetivo
    objetivo_embedding = nlp(frase_objetivo)

    mayor_similitud = -1
    frase_similar = None

    # Iterar sobre todas las frases en la lista
    for frase in lista_de_frases:
        # Crear un embedding para la frase actual
        frase_embedding = nlp(frase)

        # Calcular la similitud coseno entre la frase objetivo y la frase actual
        similitud = objetivo_embedding.similarity(frase_embedding)

        # Si la similitud es mayor que la similitud más alta encontrada hasta ahora,
        # actualizamos la similitud más alta y la frase más similar
        if similitud > mayor_similitud:
            mayor_similitud = similitud
            frase_similar = frase

    # Devolver la frase más similar y su similitud con la frase objetivo
    return frase_similar, mayor_similitud

In [4]:
encontrar_frase_similar('amo el helado de chocolate', ['amo el helado de vainilla', 'amo la ensalada de pepino'])

('amo el helado de vainilla', 0.9888834953308105)

# Ejercicio 2

Desarrolla la misma funcion, pero esta vez utilizando TF-IDF en lugar de Spacy para crear los vectores.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def encontrar_frase_similar_tfidf(frase_objetivo, lista_de_frases_input):
    # Incluir la frase objetivo en la lista de frases
    lista_de_frases = lista_de_frases_input.copy()
    lista_de_frases.append(frase_objetivo)

    # Crear el TfidfVectorizer y transformar la lista de frases
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(lista_de_frases)

    # Calcular la similitud coseno entre la frase objetivo (la última en la matriz)
    # y todas las demás frases
    similitudes = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix)

    # Obtener el índice de la frase con la mayor similitud (excluyendo la última)
    indice_similar = similitudes.argsort()[0][-2]

    # Normalizar el puntaje de similitud a que esté entre 0 y 1
    puntaje_similar = similitudes[0, indice_similar]

    # Devolver la frase más similar y su puntaje de similitud
    return lista_de_frases[indice_similar], puntaje_similar

In [6]:
encontrar_frase_similar_tfidf('amo el helado de chocolate', ['amo el helado de vainilla', 'amo la ensalada de pepino'])

('amo el helado de vainilla', np.float64(0.6496702663128741))

# Ejercicio 3

Prueba ambas funciones con el siguiente dataset. Encuentras una diferencia en el rendimiento? A qué se debe? Cuándo sería mejor utilizar una respecto a otra?


In [7]:
oracion_objetivo = "Me gusta mucho el fútbol, mi equipo favorito es River Plate."

dataset = ["A él también le gusta mucho el fútbol, siempre lo está viendo.",
            "El deporte favorito de María es el baloncesto, y su equipo es River Plate.",
            "El fútbol es un deporte muy popular en el mundo.",
            "Nunca he entendido por qué a la gente le gusta tanto el fútbol.",
            "El helado de vainilla es mi sabor favorito.",
            "Estoy aprendiendo a tocar la guitarra."]

In [8]:
encontrar_frase_similar_tfidf(oracion_objetivo, dataset)

('El deporte favorito de María es el baloncesto, y su equipo es River Plate.',
 np.float64(0.48685930446809356))

In [10]:
encontrar_frase_similar(oracion_objetivo, dataset)

('El helado de vainilla es mi sabor favorito.', 0.6684115529060364)

# GUÍA DE ESTUDIO - EJERCICIOS DE SIMILITUD

## Preguntas y Respuestas Clave

### **Comparación de Enfoques**

**P: ¿Cuál es la diferencia principal entre spaCy y TF-IDF para similitud de oraciones?**  
R: spaCy usa embeddings pre-entrenados que capturan semántica global. TF-IDF solo considera frecuencia de palabras exactas sin contexto semántico.

**P: ¿Por qué spaCy puede encontrar similitud entre sinónimos?**  
R: Sus embeddings están entrenados para que palabras con significados similares tengan vectores cercanos, mientras que TF-IDF trata "coche" y "auto" como completamente diferentes.

**P: ¿En qué casos TF-IDF podría superar a spaCy?**  
R: Cuando necesitas similitud lexical exacta, tienes vocabulario muy específico del dominio, o cuando las palabras clave literales son más importantes que el significado.

### **Implementación Técnica**

**P: ¿Por qué en TF-IDF agregamos la oración objetivo a la lista?**  
R: Para que el vectorizador aprenda el vocabulario completo y pueda comparar todas las oraciones en el mismo espacio vectorial.

**P: ¿Qué hace `argsort()[0][-2]` en la función TF-IDF?**  
R: Obtiene el índice del segundo valor más alto de similitud (excluye la oración objetivo que sería la más similar a sí misma).

**P: ¿Cómo procesa spaCy una oración completa?**  
R: Combina los embeddings de palabras individuales (típicamente promediando) para crear un vector representativo de toda la oración.

### **Análisis de Rendimiento**

**P: ¿Qué patrón observas en el dataset de fútbol entre ambos métodos?**  
R: spaCy encuentra mejor similitud semántica profunda. TF-IDF se enfoca en palabras compartidas literalmente ("fútbol", "River Plate").

**P: ¿Cuándo usarías cada método en una aplicación real?**  
R: spaCy para chatbots, búsqueda semántica, recomendaciones. TF-IDF para búsqueda de documentos, keywords matching, clasificación por temas específicos.

### **Casos de Uso Prácticos**

**P: ¿Cómo mejorarías la función de spaCy para mayor precisión?**  
R: Usar modelos más grandes (es_core_news_lg), preprocesar texto, considerar pesos por importancia de palabras, o usar embeddings especializados del dominio.

**P: ¿Qué limitaciones tiene cada approach?**  
R: spaCy: dependiente del modelo pre-entrenado, puede perder especificidad del dominio. TF-IDF: ignora semántica, sensible al vocabulario exacto.

## Puntos Clave para Recordar

1. **spaCy = semántica global** vs **TF-IDF = frecuencia local**
2. **Embeddings capturan significado** más allá de palabras exactas
3. **TF-IDF mejor para matching literal** y vocabulario específico
4. **Preprocessing afecta ambos métodos** pero diferentemente
5. **Elección depende del caso de uso** específico
6. **Combinar ambos approaches** puede ser óptimo en algunos casos

## Consideraciones Importantes

- spaCy requiere modelos pre-descargados y más memoria
- TF-IDF es más rápido y liviano pero menos "inteligente"
- La calidad del modelo spaCy afecta directamente resultados
- TF-IDF puede funcionar mejor con preprocesamiento agresivo
- Ambos son sensibles a la longitud de las oraciones

## Conexión con Próxima Clase

Estas técnicas de similitud son la base para **aplicaciones avanzadas**: sistemas de recomendación, búsqueda semántica, y extracción de información estructurada de textos.

---
*Consejo: Prueba ambos métodos con tus propias oraciones en español argentino. ¿Cuál maneja mejor lunfardo y referencias culturales locales?*